# TP CNN PyTorch
Dans ce TP, vous allez implémenter et étudier un *réseau de neurones à convolution* (Convolutional Neural Network - CNN).

## Configuration
### Si vous utilisez un ordinateur de l'Enseirb:
#### 1) Lancer une session linux (et non pas windows)
#### 2) Aller dans "Applications", puis "Autre", puis "conda_pytorch" (un terminal devrait s'ouvrir)
#### 3) Dans ce terminal, taper la commande suivante pour lancer Spyder :  
`spyder &`  
#### 4) Configurer Spyder en suivant ces instructions [Lien configuration Spyder](https://gbourmaud.github.io/files/configuration_spyder_annotated.pdf).
### Si vous utilisez votre ordinateur personnel, il faudra installer Spyder.

# I) Opération de "convolution"


En *deep learning*, l'opération de *convolution*  ne correspond pas à la définition mathématique de la convolution. Il s'agit en réalité d'une simple inter-corrélation entre un filtre (ou plusieurs filtres) et l'image à laquelle on ajoute un scalaire appelé *biais*. 

Prenons un tenseur $X$ de taille CxMxN auquel on applique un *filtre* de taille 5x5. Les paramètres de ce *filtre* sont donc un tenseur $W$ de taille Cx5x5 et un scalaire $b$. Pour une telle opération l'équation s'écrie de la manière suivante :

$Y_{0,i,j}=\sum_{k=0}^C \sum_{m=0}^{4}  \sum_{n=0}^{4} W_{k,m,n}X_{k,i+m,j+n} + b$

où $Y$ est le tenseur de sortie.

Charger et afficher l'image [peppers.png](https://gbourmaud.github.io/files/intro_deep_learning/TP/TP_CNN/peppers.png).

In [ ]:
import matplotlib.pyplot as plt
import torch 
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import PIL.Image as Image

torch.random.manual_seed(0)

# Read an example image
I_PIL = Image.open('peppers.png')
I = transforms.ToTensor()(I_PIL)

# Visualize the input x
fig = plt.figure(1), plt.imshow(I.permute((1,2,0))), plt.pause(0.1)

Utiliser la méthode `shape` du tenseur `I` pour connaître sa taille. En PyTorch, un image est representée par un tenseur (tableau multidimensionnel) dont les dimensions sont nombre_canaux x nombre_lignes x nombres_colonnes, alors que pour la bibliothèque Matplotlib, une image doit être un tableau multidimensionnel de la forme nombre_lignes x nombre_colonnes x nombre_canaux. Lire la documentation de la fonction `permute` qui est utilisée sur la dernière ligne du code précédent.

Créer 12 filtres de taille 3x5x5 en initialisation leur coefficients aléatoirement.

In [ ]:
# Create a bank of linear filters
W = torch.randn((12, 3, 5, 5))

Remarquons que ces filtres sont stockés dans un tenseur 4D.

Afficher ces 12 filtres.

In [ ]:
fig = plt.figure(2)
plt.imshow(torchvision.utils.make_grid(W, padding=4, nrow=4, normalize=True, scale_each=True).permute(1,2,0))

Appliquer ces 12 filtres à l'image `I`.

In [ ]:
Y = F.conv2d(I.unsqueeze_(0), W, bias=None, stride=1, padding=0)

Remarquons qu'en entrée de cette opération de *convolution* le tenseur doit être de la forme taille_minibatch x nombre_canaux x nombre_lignes x nombre_colonnes.

Lire la documentation de la fonction `unsqueeze_`.

Quelle devrait être la taille du tenseur résultant de l'opération précédente ? Utiliser la méthode `shape` du tenseur `Y` pour observer sa taille.

Afficher les canaux du tenseur `Y` sous la forme d'images.

In [ ]:
fig = plt.figure(3)
plt.imshow(torchvision.utils.make_grid(Y.transpose(1,0), padding=4, nrow=4, normalize=True, scale_each=True).permute(1,2,0))

Observer les différents canaux de `Y`. En principe, plusieurs, voire la plupart, de ces canaux ont des réponses fortes au niveau des contours des objets. Rappelons que les filtres ont été initialisés aléatoirement selon une distribution gaussienne centrée. Dans ce contexte, est-il normal d'observer ces fortes réponses au niveau des contours ?

Pour le moment, l'opération de *convolution* conserve la résolution de l'image d'entrée (sauf au niveau des bords) car nous avons utilisé une *stride* de 1. Il est souvent utile de réduire la résolution en sortie d'une couche de convolution. Pour cela il suffit d'augmenter la *stride*.

In [ ]:
# Try again, downsampling the output
Y_ds = F.conv2d(I, W, bias=None, stride=16, padding=0)
fig = plt.figure(4) 
plt.imshow(torchvision.utils.make_grid(Y_ds.transpose(1,0), padding=0, nrow=4, normalize=True, scale_each=True).permute(1,2,0)) 

Jusqu'à présent, l'opération de *convolution* utilisée avec une *stride* de 1 produit des effets aux bords, ce qui rend le tenseur en sortie plus petit d'un nombre de lignes et de colonnes dépendant de la taille des filtres. S'il est important d'éviter cela, le tenseur d'entrée peut être complété par des zéros (option *padding*).

In [ ]:
# Try padding 
Y_pad = F.conv2d(I, W, bias=None, stride=1, padding=2)
fig = plt.figure(5)
plt.imshow(torchvision.utils.make_grid(Y_pad.transpose(1,0), padding=0, nrow=4,
normalize=True, scale_each=True).permute(1,2,0))

Le tenseur `Y_pad` a-t-il la même taille que le tenseur d'entrée `I` ? Visualiser `Y_pad` et expliquer le résultat obtenu aux bords.

**En pratique, il est rare d'utiliser F.conv2d pour implémenter une couche de convolution car il s'agit d'une fonction. Par conséquent, les paramètres (filtres et biais) doivent être passés en argument à chaque utilisation de la fonction et gérés par l'utilisateur. On privilégiera l'implémentation d'une couche de convolution du paquet `torch.nn` : `nn.Conv2d`. Il s'agit de l'équivalent de la couche "Fully Connected" `nn.Linear` utilisée dans le TP précédent. Ainsi `nn.Conv2d` n'est pas une fonction mais une classe (programmation orientée objet). Une couche de convolution doit donc être instanciée `convLayer = nn.Conv2d(n_channels, out_channels, kernel_size, stride=1, padding=0)` afin d'obtenir un objet `convLayer`. Les paramètres (filtres et biais) peuvent être accédés de la manière suivante : `convLayer.weight` et `convLayer.bias`. La couche de convolution peut être appliquée à un tenseur de la manière suivante : `Y = convLayer(X)`.**

# II) Implémentation d'un CNN

Le code suivant permet de définir l'architecture d'un CNN.

In [ ]:
#%% CNN
class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=2, padding=2),
            nn.ReLU())
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=2, padding=2),
            nn.ReLU())
        self.fc = nn.Linear(7*7*32, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

**Faire un dessin du réseau en indiquant la taille de chaque tenseur en entrée et en sortie de chaque fonction.**

Le code suivant permet d'instancier un CNN (sur GPU si une carte compatible est disponible, sinon sur CPU) et de compter le nombre de paramètres à optimiser que contient ce CNN.

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = CNN(10).to(device)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print('Number of parameters = {}'.format(count_parameters(model)))

**Sur feuille, calculer le nombre de paramètres de chaque couche du CNN et comparer le résultat aux 28938 paramètres que le code a obtenu.**

**Lancer un apprentissage sur la base de données MNIST (attention le CNN attend en entrée un tenseur 4D et non pas un tenseur 3D comme c'était le cas du MLP). Vous devriez rapidement constater un taux de bonne classification de plus de 97%.**

Comme vu en cours, vous pouvez remplacer la dernière partie de ce CNN (qui vectorise le tenseur puis applique une couche FC) par une couche de convolution. Le résultat devrait être identique.

**Lancer un apprentissage sur la base de données MNISTTranslation (générée lors du TP précédent). Vous devriez obtenir de meilleurs résultats par rapport à ceux obtenus avec le MLP. Pouvez-vous expliquer cette amélioration des performances ? Indice : une couche de *convolution* est équivariante par translation, c'est-à-dire que si l'entrée est translatée, la sortie est également translatée.**

# (optionnel) III) Champ récepteur (Receptive field)

Considérons le précédent CNN. Intéressons-nous au tenseur `X` issu de la deuxième couche de convolution. La valeur d'un pixel de `X`, par exemple le pixel situé à la 5ème ligne et 6ème colonne (`X[1,1,4,5]`), a été obtenu en réalisant des opérations impliquant, non pas la totalité de l'image d'entrée `I`, mais une région $R$ de `I`. C'est-à-dire que les pixels de `I` situés en dehors de cette région $R$ n'interviennent pas dans le calcul de `X[1,1,4,5]`.

La taille de cette région $R$ s'appelle le [*champ récepteur*](https://distill.pub/2019/computing-receptive-fields/) (Réceptive Field an anglais).

## 1) Formule du champ récepteur d'une couche de convolution
L'objectif de cette partie est de trouver l'expression du champ récepteur $c$ d'une couche de convolution. Cette formule dépend de la taille $k$ des filtres et de la valeur de la *stride* $s$. 

Question simple : Quelle est la valeur de $c$ lorsque $s=1$ ?

Question plus difficile : Quelle est la valeur de $c$ lorsque la valeur de $s$ est quelconque ?





## 2) Calcul expérimental du champ récepteur d'une couche de convolution
Il existe plusieurs manières de calculer expérimentalement le champ récepteur d'un CNN. Ici nous allons utiliser une méthode faisant appel au calcul de la dérivée.

Commençons par le cas d'un seule couche de convolution.

Prenons le cas d'un tenseur `X` issu d'une couche de convolution prenant en entrée un tenseur `I`. La valeur d'un pixel $p$ de `X`, par exemple le pixel $p=[6,4]$ situé à la 5ème ligne et 7ème colonne (`X[0,1,4,6]`), a été obtenu en réalisant une combinaison linéaire des pixels présent dans le champ récepteur $R_p$ de `I`. 

Ainsi les pixels de `I` situés en dehors de cette région $R_p$ n'interviennent pas dans le calcul de `X[0,1,4,6]`. Cela signifie que la dérivée de `X[0,1,4,6]` par rapport à `I` vaut zéro pour tous les pixels n'appartenant pas à $R_p$.

**Travail à effectuer dans un script en PyTorch :**
- Définir une couche de convolution (`nn.Conv2d`)
- Charger une image `I` (par exemple `peppers.png`), en rajoutant une dimension (`.unsqueeze_(0)`) et en activant ses gradients (`.requires_grad_()`)
- Appliquer la convolution à cette image pour obtenir un tenseur `X`
- Calculer la dérivée d'un pixel $p$ de `X` par rapport à `I` (exemple : `X[0,1,4,6].backward()`), 
- Créer un masque des gradients (`I.grad`) dont la valeur absolue et strictement positive
- Afficher ce masque en le multipliant à `I`. Vous devriez en principe voir apparaître la région de `I` correspondant au champ récepteur du pixel $p$ de `X`.
- Vérifier que la taille de ce carré correspond à ce que vous obtenez avec la formule obtenue dans la partie précédente.
- Faire varier les paramètres de *stride* et de taille du filtre.

Lorsque le code précédent fonctionne, c'est-à-dire qu'il vous permet de calculer de champ récepteur d'une couche de convolution, vous pouvez le tester sur un réseau CNN. Récupérer l'architecture [ResNet18](https://pytorch.org/vision/main/models/generated/torchvision.models.resnet18.html) puis calculer et visualiser le champ récepteur à la sortie de chaque ResBlock.